In [19]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

##### Load dataset

In [2]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255, rotation_range=10,width_shift_range=0.1,height_shift_range=0.1,
    shear_range=0.1,zoom_range=0.10,horizontal_flip=True
)

train_generator = datagen.flow_from_directory('Dataset\Train', target_size=(150,150), batch_size=64,
                                              class_mode='binary')
valid_generator = datagen.flow_from_directory('Dataset\Val', target_size=(150,150), batch_size=64,
                                              class_mode='binary')

Found 288 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [3]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

##### Train Model

In [5]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=[150,150,3]),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 batch_normalization_3 (Bat  (None, 74, 74, 32)        128       
 chNormalization)                                                
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                        

In [6]:
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=10,
          validation_data=valid_generator)

In [7]:
model.evaluate(valid_generator)

1/1 [==============================] - 3s 3s/step - loss: 0.6934 - accuracy: 0.5167


[0.6934172511100769, 0.5166666507720947]

In [9]:
plt.plot(history.history['loss'], c='b', label='train')
plt.plot(history.history['val_loss'], c='r', label='val')

plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:
plt.plot(history.history['accuracy'], c='b', label='train')
plt.plot(history.history['val_accuracy'], c='r', label='val')

plt.legend()
plt.show()

##### Test model

In [10]:
load_model = tf.keras.models.load_model('X_Ray.h5')

In [38]:
normal_test = cv2.imread('./test/Normal2.jpeg')
normal_test = cv2.resize(normal_test, (150, 150))
normal_test = np.expand_dims(normal_test, axis=0)
normal_test = normal_test / 255
normal_test.shape

(1, 150, 150, 3)

In [39]:
pred = load_model.predict(normal_test)

1/1 [==============================] - 0s 95ms/step


In [40]:
TH = 0.5
prob = 1 if pred > TH else 0
prob

1

In [44]:
covid_test = cv2.imread('./test/SARS2.jpeg')
covid_test = cv2.resize(covid_test, (150, 150))
covid_test = np.expand_dims(covid_test, axis=0)
covid_test = covid_test / 255
covid_test.shape

(1, 150, 150, 3)

In [45]:
pred = load_model.predict(covid_test)

1/1 [==============================] - 0s 115ms/step


In [46]:
TH = 0.5
prob = 1 if pred > TH else 0
prob

0